In [26]:
# Importing the required classes
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import StreamingKMeans
from pyspark.streaming import StreamingContext

In [27]:
ssc = StreamingContext(sc,1) 

In [28]:
train_Data = sc.textFile("data.txt") 
train_Data2 = train_Data.map(lambda s:eval(s)) 
train_Data3 = train_Data2.map(lambda x:Vectors.dense(x))

In [29]:
test_Data = sc.textFile("testdata.txt")
test_Data2 = test_Data.map(lambda s:eval(s)) 
test_Data3 = test_Data2.map(lambda x:Vectors.dense(x)) 

In [30]:
trainingQueue = [train_Data3] #a list of RDD 
testingQueue = [test_Data3] #a list of RDD 

In [31]:
trainingStream = ssc.queueStream(trainingQueue)
testingStream = ssc.queueStream(testingQueue) 

In [32]:
trainingStreamWindowed = trainingStream.window(10000) # training stream windowed 10000

In [33]:
#CREATING MODEL
model = StreamingKMeans(k=20 , decayFactor = 0.8 , timeUnit = 'points')

In [34]:
centersRDD = sc.textFile('centers.txt') # read initial 
centersRDD2 = centersRDD.map(lambda s:eval(s)) # maping each string item into a list 

In [35]:
centersList = centersRDD2.collect() 
weightsList = [1.0 for i in range(20)] # set weight of each centers to 1.0

In [36]:
model.setInitialCenters(centersList,weights=weightsList)

In [37]:
model.trainOn(trainingStreamWindowed) # seting training of k-means model on windowed training stream

In [38]:
# function that saves clusterNumber,testDataVector in a text file
def saveClusterNumberTestingDataPoint(clusterNumbersRdd):
    z = clusterNumbersRdd.zip(test_Data2)
    z.coalesce(1).saveAsTextFile('PredictedClusterNumbersfinal')

In [39]:
result = model.predictOn(testingStream) # use trained model to predict cluster numbers 
result.foreachRDD(saveClusterNumberTestingDataPoint) 

In [40]:
ssc.start() # start streaming clustering

In [41]:
ssc.stop(stopSparkContext=False) # stop streaming clustering

In [42]:
centersList = list(model.latestModel().clusterCenters) # Get a list of centers 
centersList2 = [list(x) for x in centersList] # Convert each array into a list
indexList = [i for i in range(20)] # create index list for centers. It ranges from 0 to 19 

In [43]:
indexesCentersList = zip(indexList,centersList2) # zip the 2 lists so that each index gets associated with a center

In [24]:
indexesCentersRDD = sc.parallelize(indexesCentersList) # create RDD of zipped list

In [44]:
indexesCentersRDD.coalesce(1).saveAsTextFile('ClusterCenterfinal') # save the indexes and centers as a text file